### ESCENARIO PRINCIPAL

In [7]:
import pandas as pd
import pickle
import math
import datetime as dt


def calculo_nomina(salario_actual, años_hasta_jubilación, ipc):
    redondeado = math.floor(años_hasta_jubilación)
    salario = salario_actual
    for i in range(0, redondeado):
        salario = salario + salario * ipc.iloc[i, 1]
    return float(salario)

def calcular_monto_mensual(capital, intereses):
    sum=0
    for i in range(len(intereses)):
        num= 1
        for j in range(0, len(intereses[:i+1])):
            num*=(1+intereses[j])**(-1)
        sum+= num
    
    monto = capital/sum
    return monto

def calcular_primas_jubilacion_df(df, ipc, interes1, interes2, duracion_interes1, interes_rendimiento1, interes_rendimiento2, duracion_interes_rendimiento1):
    resultados = []
    with open("../../notebooks/Data Mining/modelo.pkl", "rb") as f:
        modelo = pickle.load(f)
    X_2 = df.drop(['ID', 'FECHA NAC', 'SEXO', 'FECHA ENTRADA', 'PARA CONTAR MESES'], axis=1)
    X_2.columns = ['Ingreso Anual (€)','Edad']
    X_2 = X_2[['Edad','Ingreso Anual (€)']] 
    df["AÑOS HASTA JUBILACION"] = modelo.predict(X_2)
    for _, row in df.iterrows():
        salario_final = calculo_nomina(row['NOMINA BRUTA 01/01/2025'], row['AÑOS HASTA JUBILACION'], ipc)
        porcentaje_renta = min((row['EDAD'] // 4) * 0.0225, 0.19)
        m1 = (salario_final * porcentaje_renta) / 12
        
        fecha_jubilacion = dt.datetime(2025, 1, 1) + dt.timedelta(days=row['AÑOS HASTA JUBILACION'] * 365)
        fecha_jubilacion = fecha_jubilacion.strftime('%Y-%m-%d')
        date_range = pd.date_range(start=fecha_jubilacion, periods=22 * 12, freq='MS')
        date_list = date_range.strftime('%Y-%m').tolist()
        
        int1 = ((1 + (interes1 * 0.01)) ** (1 / 12)) - 1
        int2 = ((1 + (interes2 * 0.01)) ** (1 / 12)) - 1
        año1, mes1 = duracion_interes1
        
        intereses = [int1] * (año1 * 12 + mes1) + [int2] * ((22 * 12) - (año1 * 12 + mes1))
        
        rentas = []
        valor = m1
        for fecha in date_list:
            if fecha[-2:] == '01':
                valor *= 1.03
            rentas.append(valor)
        
        df_rentas = pd.DataFrame({'Fecha': date_list, 'Intereses': intereses, 'Pagos': rentas})
        
        capital_jubilacion = 0
        for i in range(len(df_rentas) - 1, -1, -1):
            renta = float(df_rentas.iloc[i, 2]) * (1 + float(df_rentas.iloc[i, 1])) ** -1
            for interes in range(i, -1, -1):
                renta = renta * (1 + float(df_rentas.iloc[interes, 1])) ** -1
            capital_jubilacion += renta
        
        fechas = pd.date_range(start='2025-01-01', end=fecha_jubilacion, freq='MS')
        fechas_formateadas = fechas.strftime('%Y-%m')
        
        int_ren1 = ((1 + (interes_rendimiento1 * 0.01)) ** (1 / 12)) - 1
        int_ren2 = ((1 + (interes_rendimiento2 * 0.01)) ** (1 / 12)) - 1
        año2, mes2 = duracion_interes_rendimiento1
        
        intereses1 = [int_ren1] * (año2 * 12 + mes2) + [int_ren2] * (len(fechas_formateadas) - (año2 * 12 + mes2))
        df1 = pd.DataFrame({'Fecha': fechas_formateadas, 'Intereses': intereses1})
        
        capital_actual = capital_jubilacion
        for i in range(len(df1) - 1, -1, -1):
            capital_actual = capital_actual * (1 + float(df1.iloc[i, 1])) ** -1
        
        monto_mensual = calcular_monto_mensual(capital_jubilacion, df1['Intereses'].tolist())
        
        resultados.append([row['ID'], capital_jubilacion, capital_actual, monto_mensual])
    
    return pd.DataFrame(resultados, columns=['ID', 'Capital Jubilación', 'Capital Actual', 'Monto Mensual'])

# Cargar los datos y calcular resultados
df_input = pd.read_csv('../../Datos/Limpios/datos1_limpios.csv', index_col=0)
ipc_data = pd.read_excel('../../Datos/Originales/Dataset_1.xlsx', sheet_name='IPC')
resultados_df = calcular_primas_jubilacion_df(df_input, ipc_data, 2, 1.5, [6,9], 2.5, 2, [7,0])
print(resultados_df)


     ID  Capital Jubilación  Capital Actual  Monto Mensual
0     1       107929.622400    79044.634854     754.781421
1     2       170034.093333    81754.958037     575.786943
2     3       124495.870475    95174.453457    1007.618961
3     4        90971.927228    66405.763329     629.756736
4     5       146242.762248   103969.478056     937.617942
..  ...                 ...             ...            ...
95   96       176848.801468   129305.554148    1230.460491
96   97       171826.411025    91366.197089     649.686157
97   98       182500.267745    81066.404032     574.844294
98   99       242550.898824   133741.287476     960.321023
99  100       217555.431681   132225.957290     995.985954

[100 rows x 4 columns]


In [8]:
resultados_df['Capital Actual'].sum()

np.float64(10312093.356052017)

In [9]:
resultados_df['Monto Mensual'].sum()

np.float64(83525.40701110722)

In [10]:
resultados_df['Capital Jubilación'].sum()

np.float64(17555353.35504604)

### OTROS ESCENARIOS

In [11]:
import pandas as pd
import pickle
import math
import datetime as dt

def calculo_nomina(salario_actual, años_hasta_jubilación, ipc):
    redondeado = math.floor(años_hasta_jubilación)
    salario = salario_actual
    for i in range(0, redondeado):
        salario = salario + salario * ipc.iloc[i, 1]
    return float(salario)

def calcular_monto_mensual(capital, intereses):
    sum=0
    for i in range(len(intereses)):
        num= 1
        for j in range(0, len(intereses[:i+1])):
            num*=(1+intereses[j])**(-1)
        sum+= num
    
    monto = capital/sum
    return monto

def extender_lista(lista, tamaño):
    if len(lista) >= tamaño:
        return lista[:tamaño]
    else:
        return (lista * (tamaño // len(lista) + 1))[:tamaño]

def calcular_primas_jubilacion_df(df, ipc, lista_intereses):
    resultados = []
    with open("../../notebooks/Data Mining/modelo.pkl", "rb") as f:
        modelo = pickle.load(f)
    X_2 = df.drop(['ID', 'FECHA NAC', 'SEXO', 'FECHA ENTRADA', 'PARA CONTAR MESES'], axis=1)
    X_2.columns = ['Ingreso Anual (€)','Edad']
    X_2 = X_2[['Edad','Ingreso Anual (€)']]
    df["AÑOS HASTA JUBILACION"] = modelo.predict(X_2)
    for _, row in df.iterrows():
        salario_final = calculo_nomina(row['NOMINA BRUTA 01/01/2025'], row['AÑOS HASTA JUBILACION'], ipc)
        porcentaje_renta = min((row['EDAD'] // 4) * 0.0225, 0.19)
        m1 = (salario_final * porcentaje_renta) / 12
        
        fecha_jubilacion = dt.datetime(2025, 1, 1) + dt.timedelta(days=row['AÑOS HASTA JUBILACION'] * 365)
        fecha_jubilacion = fecha_jubilacion.strftime('%Y-%m-%d')
        date_range = pd.date_range(start=fecha_jubilacion, periods=22 * 12, freq='MS')
        date_list = date_range.strftime('%Y-%m').tolist()
        
        intereses = extender_lista(lista_intereses, len(date_list))
        
        rentas = []
        valor = m1
        for fecha in date_list:
            if fecha[-2:] == '01':
                valor *= 1.03
            rentas.append(valor)
        
        df_rentas = pd.DataFrame({'Fecha': date_list, 'Intereses': intereses, 'Pagos': rentas})
        
        capital_jubilacion = 0
        for i in range(len(df_rentas) - 1, -1, -1):
            renta = float(df_rentas.iloc[i, 2]) * (1 + float(df_rentas.iloc[i, 1])) ** -1
            for interes in range(i, -1, -1):
                renta = renta * (1 + float(df_rentas.iloc[interes, 1])) ** -1
            capital_jubilacion += renta
        
        fechas = pd.date_range(start='2025-01-01', end=fecha_jubilacion, freq='MS')
        fechas_formateadas = fechas.strftime('%Y-%m')
        
        intereses1 = extender_lista(lista_intereses, len(fechas_formateadas))
        df1 = pd.DataFrame({'Fecha': fechas_formateadas, 'Intereses': intereses1})
        
        capital_actual = capital_jubilacion
        for i in range(len(df1) - 1, -1, -1):
            capital_actual = capital_actual * (1 + float(df1.iloc[i, 1])) ** -1
        
        monto_mensual = calcular_monto_mensual(capital_jubilacion, df1['Intereses'].tolist())
        
        resultados.append([row['ID'], capital_jubilacion, capital_actual, monto_mensual])
        resultados_df = pd.DataFrame(resultados, columns=['ID', 'Capital Jubilación', 'Capital Actual', 'Monto Mensual'])
    
    return resultados_df

### SP500

In [12]:
df_input = pd.read_csv('../../Datos/Limpios/datos1_limpios.csv', index_col=0)
ipc_data = pd.read_excel('../../Datos/Originales/Dataset_1.xlsx', sheet_name='IPC')
intereses_lista = pd.read_csv('../../Datos/Limpios/datos1_limpios.csv', index_col=0)
intereses_lista = pd.read_csv('../../Datos/Intereses/Datos históricos del S&P 500.csv', index_col=0)
lista=list(intereses_lista['% var.'])
intereses_lista = [float(str(i).replace('%', '').replace(',', '.')) / 100 for i in lista]

resultados_df = calcular_primas_jubilacion_df(df_input, ipc_data, intereses_lista)
print(resultados_df)


     ID  Capital Jubilación  Capital Actual  Monto Mensual
0     1        48186.160343    11271.687573     559.262923
1     2        75895.912642     4731.424973     592.557296
2     3        55571.855778    15987.780240     695.657574
3     4        40607.920704     9214.582508     468.798802
4     5        65274.938021    12174.246245     727.099211
..  ...                 ...             ...            ...
95   96        78947.271704    18487.650995     913.801890
96   97        76699.631264     6365.166951     620.096775
97   98        81460.277209     3035.687248     624.570896
98   99       108254.498618    14456.320585     891.954722
99  100        97099.907430    18153.115842     889.160771

[100 rows x 4 columns]


### IBEX35

In [13]:
df_input = pd.read_csv('../../Datos/Limpios/datos1_limpios.csv', index_col=0)
ipc_data = pd.read_excel('../../Datos/Originales/Dataset_1.xlsx', sheet_name='IPC')
intereses_lista = pd.read_csv('../../Datos/Limpios/datos1_limpios.csv', index_col=0)
intereses_lista = pd.read_csv('../../Datos/Intereses/Datos históricos del IBEX 35.csv', index_col=0)
lista=list(intereses_lista['% var.'])
intereses_lista = [float(str(i).replace('%', '').replace(',', '.')) / 100 for i in lista]

resultados_df = calcular_primas_jubilacion_df(df_input, ipc_data, intereses_lista)
print(resultados_df)

     ID  Capital Jubilación  Capital Actual  Monto Mensual
0     1        99939.661060    80816.163022     836.108610
1     2       157446.791129    24969.447072     591.968354
2     3       115273.976072    73333.893788    1121.578108
3     4        84235.670911    69599.875236     695.097453
4     5       135402.894828   121722.578285    1009.560980
..  ...                 ...             ...            ...
95   96       163748.290635   135852.039913    1360.497124
96   97       159103.071177    36990.744142     628.238700
97   98       168990.118237    25697.930002     619.864710
98   99       224580.307699    80160.672116     908.631883
99  100       201440.171542   119743.854801     974.648608

[100 rows x 4 columns]


### BONOS ESTADOS UNIDOS

In [14]:
df_input = pd.read_csv('../../Datos/Limpios/datos1_limpios.csv', index_col=0)
ipc_data = pd.read_excel('../../Datos/Originales/Dataset_1.xlsx', sheet_name='IPC')
intereses_lista = pd.read_csv('../../Datos/Limpios/datos1_limpios.csv', index_col=0)
intereses_lista = pd.read_csv('../../Datos/Intereses/Estados Unidos Datos Históricos Rendimiento de Bonos.csv', index_col=0)
lista=list(intereses_lista['% var.'])
intereses_lista = [float(str(i).replace('%', '').replace(',', '.')) / 100 for i in lista]

resultados_df = calcular_primas_jubilacion_df(df_input, ipc_data, intereses_lista)
print(resultados_df)

     ID  Capital Jubilación  Capital Actual  Monto Mensual
0     1        96334.461709    77519.077703    1013.479374
1     2       151775.848762   279200.806246     365.620209
2     3       111133.965599    54944.232853    1347.369964
3     4        81201.786025    63537.619917     840.343776
4     5       130553.212867   100074.042703    1203.116048
..  ...                 ...             ...            ...
95   96       157841.095124   125259.032367    1646.806431
96   97       153372.714931   269583.505200     491.179478
97   98       162903.406566   339377.797263     320.772972
98   99       216519.372666   303719.298756     775.683911
99  100       194229.173604   191154.535443     965.569741

[100 rows x 4 columns]


### Euro Stoxx 50

In [15]:
df_input = pd.read_csv('../../Datos/Limpios/datos1_limpios.csv', index_col=0)
ipc_data = pd.read_excel('../../Datos/Originales/Dataset_1.xlsx', sheet_name='IPC')
intereses_lista = pd.read_csv('../../Datos/Limpios/datos1_limpios.csv', index_col=0)
intereses_lista = pd.read_csv('../../Datos/Intereses/Datos históricos del Euro Stoxx 50.csv', index_col=0)
lista=list(intereses_lista['% var.'])
intereses_lista = [float(str(i).replace('%', '').replace(',', '.')) / 100 for i in lista]

resultados_df = calcular_primas_jubilacion_df(df_input, ipc_data, intereses_lista)
print(resultados_df)

     ID  Capital Jubilación  Capital Actual  Monto Mensual
0     1        81570.034994    44370.450677     751.411434
1     2       128509.771113    30648.511705     600.729152
2     3        94083.769736    48685.965874     976.854471
3     4        68754.451672    37948.899361     626.916975
4     5       110524.340307    59338.446134     937.169202
..  ...                 ...             ...            ...
95   96       133652.681907    75252.128260    1224.836471
96   97       129862.376590    35050.482972     655.563706
97   98       137931.559349    53167.867244     607.638677
98   99       183306.702111    55583.832897     957.553898
99  100       164422.962044    61945.735265     953.872790

[100 rows x 4 columns]
